# K-Means

In [18]:
import numpy as np
import scipy.io as sio
import scipy.spatial.distance as scidist
from plotly.offline import plot, iplot, init_notebook_mode
from plotly.graph_objs import Scatter3d, Layout

In [2]:
data = sio.loadmat('./data/toydata2.mat')['data']

In [3]:
#   % This function should assign the points in your dataMatrix to the
#    % closest centroid, the distance is computed using the L2 norm
#        %
#        % Input:
#        %       dataMatrix (nrDims x nrSamples)
#        %       centroids (nrDims x nrCentroids)
#        % Output
#        %       assigedPoints (1 x nrSamples) should have the centroid's
#        %       index


def assignPoints(dataMatrix, centroids):
    nrSamples = dataMatrix.shape[1]
    allDists = scidist.cdist(dataMatrix.transpose(), centroids.transpose())
    assignedPoints = np.argmin(allDists, 1)
    
    # return the assigned points    
    return assignedPoints

In [4]:
#    % This function should assign the points in your dataMatrix to the
#    % closest centroid, the distance is computed using the L2 norm
#        %
#        % Input:
#        %       dataMatrix (nrDims x nrSamples)
#        %       assigedPoints (1 x nrSamples)
#        % Output
#        %       updatedCentroids (nrDims x nrCentroids)


def updateCentroids(dataMatrix, assignedPoints):
    centroids = np.unique(assignedPoints).astype(int)
    nrDims = dataMatrix.shape[0]    
    updatedCentroids = np.zeros((nrDims, len(centroids)))
    for c in centroids:
        idxs = np.where(assignedPoints == c)
        points = dataMatrix[:,idxs[0]]
        updatedCentroids[:,c] = np.mean(points, 1)
    # return the updated centroids
    return updatedCentroids


In [5]:
#   % This function should compute the cost function
#    %
#        % Input:
#        %       dataMatrix (nrDims x nrSamples)
#        %       centroids (nrDims x nrCentroids)
#        %       assignedPoints (1 x nrSamples)
#        % Output
#        %       cost


def computeCost(dataMatrix, centroids, assignedPoints):
    cost = 0
    nrDims = dataMatrix.shape[0]    
    updatedCentroids = np.zeros((nrDims, len(centroids)))
    for c in range(centroids.shape[1]):
        idxs = np.where(assignedPoints == c)
        points = dataMatrix[:,idxs[0]]
        distances = scidist.cdist(dataMatrix.transpose(), np.array([centroids[:,c].transpose()]))
        cost += sum(distances)[0]
    cost /= dataMatrix.shape[1]
    return cost


In [6]:

#    % This function should run the K-Means algorithm
#    %
#        % Input:
#        %       dataMatrix (nrDims x nrSamples)
#        %       numberOfCluster
#        %       numberOfRuns
#        % Output
#        %       object
def runKmeans(dataMatrix, numberOfCluster, numberOfRuns = 1):

    
#    % You can neglect the number of runs at the begining, once you
#    % are done, you can work on it. The idea is to run the Kmeans
#    % several times (runs), then choose the one giving you the min.
#    % cost.
            
    nrDim = dataMatrix.shape[0]
    nrSamples = dataMatrix.shape[1]
    
    costs = np.zeros(numberOfRuns)
    results = np.zeros((numberOfRuns, nrSamples))

    for i in range(numberOfRuns):
#         % initializa Centroids
        min_, max_ = np.amin(dataMatrix, 1), np.amax(dataMatrix, 1)
        centroids = np.random.rand(nrDim, numberOfCluster)
        centroids = (np.multiply(centroids.transpose(), (max_-min_)) + min_).transpose()
        print (centroids)
        assignedPoints = assignPoints(dataMatrix, centroids)
        prevAssignedPoints = np.zeros(nrSamples)
#         print (centroids, assignedPoints)
        while sum(assignedPoints - prevAssignedPoints):
            centroids = updateCentroids(dataMatrix, assignedPoints.transpose())
            prevAssignedPoints = assignedPoints
            assignedPoints = assignPoints(dataMatrix, centroids)
        
        costs[i] = computeCost(data, centroids, assignedPoints)
        results[i] = assignedPoints
            
    minidx = np.argmin(costs)
    
    return results[minidx]

In [19]:
def plot3D(x,y,z,color):
    data_plt = Scatter3d(
        x=x, 
        y=y, 
        z=z,
        mode='markers',
        marker=dict(
            size=5,
            color=color,                
            colorscale='Viridis',   
            opacity=0.8
        )
    )
    return data_plt


def plotClusters(assignedPoints):
    idxs0 = np.where(assignedPoints == 0)
    data0 = data[:,idxs0[0]]
    idxs1 = np.where(assignedPoints == 1)
    data1 = data[:,idxs1[0]]
    idxs2 = np.where(assignedPoints == 2)
    data2 = data[:,idxs2[0]]

    p0 = plot3D(
        x=data0[0], 
        y=data0[1], 
        z=data0[2],
        color='blue'
    )

    p1 = plot3D(
        x=data1[0], 
        y=data1[1], 
        z=data1[2],
        color='red'
    )

    p2 = plot3D(
        x=data2[0], 
        y=data2[1], 
        z=data2[2],
        color='green'
    )

    iplot([p0, p1, p2])


In [23]:

#        % This function should run the K-Means algorithm for visualization
#        % Input:
#        %       dataMatrix (nrDims x nrSamples)
#        %       numberOfCluster
#        % Output
#        %       object
    

def runKmeansVis(dataMatrix, numberOfCluster):
    nrDim = dataMatrix.shape[0]
    nrSamples = dataMatrix.shape[1]
    min_, max_ = np.amin(dataMatrix, 1), np.amax(dataMatrix, 1)
    centroids = np.random.rand(nrDim, numberOfCluster)
    centroids = (np.multiply(centroids.transpose(), (max_-min_)) + min_).transpose()
    print (centroids)
    assignedPoints = assignPoints(dataMatrix, centroids)
    prevAssignedPoints = np.zeros(nrSamples)
    while sum(assignedPoints - prevAssignedPoints):
        centroids = updateCentroids(dataMatrix, assignedPoints.transpose())
        prevAssignedPoints = assignedPoints
        assignedPoints = assignPoints(dataMatrix, centroids)
        plotClusters(assignedPoints)
        

       

In [24]:
# testing kmeans

init_notebook_mode(connected=True)

assignedPoints = runKmeans(data, 3)
plotClusters(assignedPoints)

[[-1.00146464  4.84198762 -0.57566578]
 [ 4.62491412  3.97880709  2.33792918]
 [ 1.78107199  5.53132215  6.01792089]]


In [26]:
runKmeansVis(data, 3)

[[-0.52092556  4.1169061  -2.87932433]
 [ 1.89351544  3.17103584  3.9207793 ]
 [ 5.17906037 -1.27643654 -0.19059882]]
